# Neural Networks Review and Practice Session!

Using both Text and Image data with Neural Networks!

In [1]:
# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# *OPEN IN GOOGLE COLAB IF I WANT TO RUN NEURAL NETWORKS!*

In [2]:
# #Lindsey imports this way so she doesn't have to worry about importing every function, methods, classes
# from tensorflow import keras 
# from tensorflow.keras import layers

In [3]:
# Defining a results visualization function
def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()

## Part 0: Some Resources/References/Cheat Codes for Tuning Neural Networks

<img src="https://media.giphy.com/media/st83jeYy9L6Bq/giphy.gif">

### a. Adding nodes and layers

- Number of hidden layers

> For many problems you can start with just one or two hidden layers it will work just fine. For more complex problems, you can gradually ramp up the number of hidden layers until your model starts to over fit. Very complex tasks, like image classification without convolutional layers, will need dozens of layers.

- Number of neurons per layer

> The number of neurons for the input and output layers is dependent on your data and the task. i.e. input dimensions are determined by your number or columns and your output layer for classification has just one node with a sigmoid activation function. For hidden layers, a common practice is to create a funnel with funnel with fewer and fewer neurons per layer.


In general, you will get more bang for your buck by adding on more layers than adding more neurons.

### b. Activation functions

* One thing important for activation functions is its **differentiability** because the derivative is used in the backpropagation process

<img src='images/activation.png' width=500/>

#### Activation functions for output layers (for supervised learning problems)

1. For binary classification problems: sigmoid activation to coerce values between 0-1
2. For multiclass classification: softmax activation, as it produces a non-negative vector that sums to 1 (probabilities of your test point belonging to the different classes)
3. For regression problems: linear, or relu activation (it is linear and unbounded!)

#### Activation functions for hidden layers

Relevant/useful blog posts (by different authors, despite the similar titles): 
- [Exploring Activation Functions for Neural Networks](https://towardsdatascience.com/exploring-activation-functions-for-neural-networks-73498da59b02)
    - Discusses why adding non-linear activation functions helps us solve non-linear problems, as well as why an understanding of the derivatives of these activation functions helps us tune NNs more effectively
- [Understanding Activation Functions in Neural Networks](https://medium.com/the-theory-of-everything/understanding-activation-functions-in-neural-networks-9491262884e0)
    - Builds the intuitition behind activation functions (despite less-than-stellar grammar and punctuation throughout)

Notes: 
* Sigmoids are not used often because it has a small maximum derivative value and thus propagates only a small amount of error each time, leading to slow "learning" 
* This small-derivative-slow-learning issue is known as a **vanishing gradient** problem
* Tanh is mathematically quite similar to a sigmoid function, thus also has the vanishing gradient issue, but not as bad
* ReLu generally works well because its gradient is always 1, as long as the input is positive (no vanishing gradients), and negative inputs going to 0 can make your network lighter (no weights/biases are being updated)

Here are resources on both the [vanishing gradient](https://machinelearningmastery.com/how-to-fix-vanishing-gradients-using-the-rectified-linear-activation-function/m) and [exploding gradient](https://machinelearningmastery.com/exploding-gradients-in-neural-networks/) problems. 

### c. Loss functions

Loss functions are akin to cost functions we were trying to minimize in gradient descent (i.e. RMSE for linear regression, Gini/entropy for trees)

1. For regression problems, keras has **mean_squared_error** or **mean_absolute_error** as a loss function, or **mean_squared_logarithmic_error** if your target has potential outliers
2. For binary classification: **binary_crossentropy** 
3. For multiclass problems: **categorical_crossentropy**

[This article summarizes the above, and more.](https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/)

### d. Optimizers

Overview Resource: 
- [A (Quick) Guide to Neural Network Optimizers with Applications in Keras](https://towardsdatascience.com/a-quick-guide-to-neural-network-optimizers-with-applications-in-keras-e4635dd1cca4) blog post - an overview of the options

Summary:
* Different optimizers are just different methods/paths that your neural network can take to find optimal values
* Experimentally, Adam (derived from *adaptive moment estimation*) is a good one to use - [here's more about Adam](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)

#### Quick optimizer summary

* **RMSProp**: maintains per-parameter learning rates adapted based on the average of recent weight updates (e.g. how quickly it is changing). This does well on non-stationary problems (e.g. noisy data)

* **Adagrad**: maintains a per-parameter learning rate that improves performance on problems with sparse gradients (e.g. natural language and computer vision problems)

* **Adam**: realizes the benefits of both AdaGrad and RMSProp. Instead of adapting the parameter learning rates based on the average first moment (the mean) as in RMSProp, Adam also makes use of the average of the second moments of the gradients (the uncentered variance)

### e. Learning rate

* The learning rate is something you can define when you compile your model with the optimizer
* Optimizers usually change up the learning rates, so this is just the *initial* learning rate
* If you set it too low, training will eventually converge, but it will do so slowly
* If you set it too high, it might actually diverge
* If you set it slightly too high, it will converge at first but miss the local optima

### f. Regularization

* As a neural network learns, neuron weights settle into their context within the network
* Weights of neurons are tuned for specific features providing some specialization
* Neighboring neurons become too reliant on this specialization, which if taken too far can result in a fragile model too specialized to the training data
* This reliance on context for a neuron during training is referred to as *complex co-adaptations*

#### Methods
1. You can add L1 or L2 regularization within each hidden layer
2. You can also add a **dropout layer** 
3. Not technically *regularization*, but you can introduce **early stopping** so your model doesn't overtrain

#### Dropout
Dropout is a technique where randomly selected neurons are ignored during training. They are “dropped-out” randomly. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass. You can add **dropout layers** in your neural network.

<img src='images/thanos.png'/>

# Part 1: Text Classification

## Pre-Trained Networks and Word Embeddings and LSTMs, oh my!


*Kaggle Reference for Text Classification NN: https://www.kaggle.com/watts2/glove6b50dtxt*

In [4]:
# More specific imports
import nltk
from nltk.corpus import stopwords

from gensim.models import word2vec

Data is: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [5]:
df = pd.read_csv('data/IMDB_Reviews.csv')

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [8]:
# Our target value
df['sentiment'].unique()

array(['positive', 'negative'], dtype=object)

### Pre-Split Preprocessing

Doing some initial preprocessing that can be done before the train/test split

In [9]:
# Let's check out an example review...
index_num = 15903 # Defining the index number of the review to explore

df['review'].iloc[index_num]

'THIS IS NOT A CHILDREN\'S MOVIE!!!<br /><br />This movie is like a "bad acid trip" for kids under the age of 5. For a month my 4 year old from time-to-time would ask me "Why was that rabbit bleeding from its mouth" or "Why did the bulldozer bury all the rabbits?". (And that wasn\'t the worst of it). We stopped it about a 1/2 hour in but the damage had been done. Intensely morbid, oppressive, violent. Fortunately he\'s finally forgotten about the whole wretched thing. Whomever decided this movie should be marketed to children should be brought up on charges. ... (Go ahead censure me, my conscience is clear.).'

We have some HTML tags inside these texts... will want to remove them. But how?

Enter: Regular Expressions (regex).

Testing: https://regexr.com/

In [10]:
# Find the pattern to remove html tags
import re

html_tag_pattern = re.compile(r'<[^>]*>')

test = html_tag_pattern.sub('', df['review'].iloc[index_num])

In [11]:
test

'THIS IS NOT A CHILDREN\'S MOVIE!!!This movie is like a "bad acid trip" for kids under the age of 5. For a month my 4 year old from time-to-time would ask me "Why was that rabbit bleeding from its mouth" or "Why did the bulldozer bury all the rabbits?". (And that wasn\'t the worst of it). We stopped it about a 1/2 hour in but the damage had been done. Intensely morbid, oppressive, violent. Fortunately he\'s finally forgotten about the whole wretched thing. Whomever decided this movie should be marketed to children should be brought up on charges. ... (Go ahead censure me, my conscience is clear.).'

In [12]:
# Apply our pattern to the dataset
df['review'] = df['review'].map(lambda x: re.sub(r'<[^>]*>', '', x))

# Same as
# df['review'] = df['review'].map(lambda x: html_tag_pattern.sub('', x))

In [13]:
# Sanity check
df['review'].iloc[index_num]

'THIS IS NOT A CHILDREN\'S MOVIE!!!This movie is like a "bad acid trip" for kids under the age of 5. For a month my 4 year old from time-to-time would ask me "Why was that rabbit bleeding from its mouth" or "Why did the bulldozer bury all the rabbits?". (And that wasn\'t the worst of it). We stopped it about a 1/2 hour in but the damage had been done. Intensely morbid, oppressive, violent. Fortunately he\'s finally forgotten about the whole wretched thing. Whomever decided this movie should be marketed to children should be brought up on charges. ... (Go ahead censure me, my conscience is clear.).'

Let's also remove stopwords

In [14]:
stop_words = stopwords.words('english')

In [15]:
# Neat bit of code!
df['review'] = df['review'].apply(lambda x: ' '.join(
    [word for word in x.split() if word.lower() not in (stop_words)]))

In [16]:
# Sanity Check
df['review'].iloc[index_num]

'CHILDREN\'S MOVIE!!!This movie like "bad acid trip" kids age 5. month 4 year old time-to-time would ask "Why rabbit bleeding mouth" "Why bulldozer bury rabbits?". (And worst it). stopped 1/2 hour damage done. Intensely morbid, oppressive, violent. Fortunately he\'s finally forgotten whole wretched thing. Whomever decided movie marketed children brought charges. ... (Go ahead censure me, conscience clear.).'

Can also pre-process our target variable

In [17]:
# Create a target map
target_map = {'positive': 1,
              'negative': 0}

In [18]:
# Map it
df['sentiment'] = df['sentiment'].map(target_map)

In [19]:
# Sanity check
df.head()

,review,sentiment
0,One reviewers mentioned watching 1 Oz episode ...,1
1,wonderful little production. filming technique...,1
2,thought wonderful way spend time hot summer we...,1
3,Basically there's family little boy (Jake) thi...,0
4,"Petter Mattei's ""Love Time Money"" visually stu...",1


### Split, and then Post-Split Processing
Now let's perform a train/test split:

In [20]:
# Define our X and y
X = df['review']
y = df['sentiment']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [21]:
X_train.shape

(37500,)

In [22]:
# Need to find that same review now that the index is shuffled
train_index_num = X_train.index.get_loc(15903)
X_train.iloc[train_index_num]

'CHILDREN\'S MOVIE!!!This movie like "bad acid trip" kids age 5. month 4 year old time-to-time would ask "Why rabbit bleeding mouth" "Why bulldozer bury rabbits?". (And worst it). stopped 1/2 hour damage done. Intensely morbid, oppressive, violent. Fortunately he\'s finally forgotten whole wretched thing. Whomever decided movie marketed children brought charges. ... (Go ahead censure me, conscience clear.).'

### Vanilla Text Classification... What Would We Do?

Aka what would this look like without a NN?

In [23]:
# Let's use a TF-IDF vectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
# What parameters should we set? What steps have we already done, what do we still need to do?
# Already removed stopwords!
vectorizer = TfidfVectorizer(
    max_df=.95,  # removes words that appear in more than 95% of docs
    min_df=2 # removes words that appear 2 or fewer times
    #This is dimensionality reduction
)  

In [25]:
vectorizer.fit(X_train)

X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

#### Explore Our Vectorized Text

In [26]:
# Let's look at that second example again
X_train.iloc[train_index_num]

'CHILDREN\'S MOVIE!!!This movie like "bad acid trip" kids age 5. month 4 year old time-to-time would ask "Why rabbit bleeding mouth" "Why bulldozer bury rabbits?". (And worst it). stopped 1/2 hour damage done. Intensely morbid, oppressive, violent. Fortunately he\'s finally forgotten whole wretched thing. Whomever decided movie marketed children brought charges. ... (Go ahead censure me, conscience clear.).'

In [27]:
train_index_num

22171

In [28]:
X_train.loc[X_train.str.contains('CHILDREN\'S MOVIE!!!')]

15903    CHILDREN'S MOVIE!!!This movie like "bad acid t...
Name: review, dtype: object

In [29]:
# Creating a df of tf-idf values, where each column is a word in the vocabulary
tfidf_train_df = pd.DataFrame(X_train_vec.toarray(), 
                              columns=vectorizer.get_feature_names(), 
                              index=X_train.index)

In [30]:
# Grabbing that row once it's been vectorized
test_doc = tfidf_train_df.iloc[train_index_num]

test_doc[test_doc > 0].sort_values(ascending=False).head(15) # Showing values > 0

censure       0.248884
bulldozer     0.228991
why           0.217933
whomever      0.202968
children      0.202369
rabbits       0.197874
oppressive    0.188972
bleeding      0.186441
bury          0.185497
charges       0.183285
marketed      0.182048
intensely     0.174200
morbid        0.174200
wretched      0.170670
conscience    0.168263
Name: 15903, dtype: float64

What does this tell you about the word "censure" in the this document?

- Censure is rarer than bulldozer in other documents


In [31]:
# Now let's model
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [32]:
classifier.fit(X_train_vec, y_train)

classifier.score(X_test_vec, y_test)

0.86592

Evaluate:

- Not bad


In [33]:
#Baseline Understanding
y_train.value_counts(normalize=True)

0    0.50248
1    0.49752
Name: sentiment, dtype: float64

*We can consider our baseline model to be 86% accurate*

### Moving to NN-Based Text Classification!

#### Different Pre-Processing Steps!

Let's walk through these steps first, then discuss why we didn't just use vectorized text.

Going to use keras's tokenizer: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
# Find our longest review - will need for padding later
max_length = max([len(s.split()) for s in X_train])
max_length

In [ ]:
# Now to tokenize
# Recommend checking out their default values - they're removing punctuation for us!
tokenizer = keras.preprocessing.text.Tokenizer()

tokenizer.fit_on_texts(X_train)

X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [ ]:
#Most common words not removed
tokenizer.word_index 

In [ ]:
# What is this doing? 
#Let's look at the first 10 key-value pairs in the word_index dict

list(tokenizer.word_index.items())[:10]

In [ ]:
# Same example, after processing
print(X_train_token[train_index_num])

In [ ]:
# Grab the corpus size
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

In [ ]:
# Now, let's pad so each review is the same length as our longest review
# Basically, adding zeros at the end

X_train_processed = keras.preprocessing.sequence.pad_sequences(
    X_train_token, maxlen=max_length, padding='post')
X_test_processed = keras.preprocessing.sequence.pad_sequences(
    X_test_token, maxlen=max_length, padding='post')

In [ ]:
print(X_train_processed[2])

#### Why Couldn't We Just Use TF-IDF?

In [ ]:
# Look at the vectorized text
tfidf_train_df.head()

In [ ]:
# Look at the preprocessed text from keras
X_train_processed

What is the difference? Specifically, what are the columns representing in each of these? What are the numbers?

- In `tfidf_train_df`, each column is a word in our Corpus
- In `X_train_processed`, each column is the placement of the word in the sentence. It's value is the rank of the word in the dictionary. For example, every time `1` shows up, that means `movie`.


## Now: Using Pre-Trained Embeddings and NNs for NLP Tasks

For the most part, following this example: https://stackabuse.com/python-for-nlp-movie-sentiment-analysis-using-deep-learning-in-keras/

Also relevant: https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/

### Pre-Trained Word Embeddings: GloVe (Global Vectors for Word Representation)

The link to download the GloVe files: https://nlp.stanford.edu/projects/glove/

> **You will need to download the GloVe embeddings directly, since these files are all too big for github!**

The below function and code comes from: https://realpython.com/python-keras-text-classification/#using-pretrained-word-embeddings

*Lindsey didn't write the following code, but it works*

In [ ]:
def create_embedding_matrix(glove_filepath, word_index, embedding_dim):
    '''
    Grabs the embeddings just for the words in our vocabulary
    
    Inputs:
    glove_filepath - string, location of the glove text file to use
    word_index - word_index attribute from the keras tokenizer
    embedding_dim - int, number of dimensions to embed, a hyperparameter
    
    Output:
    embedding_matrix - numpy array of embeddings
    '''
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(glove_filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [ ]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('glove.6B.50d.txt',
                                           tokenizer.word_index, 
                                           embedding_dim)

In [ ]:
#Number of words in our vocabulary, by number of dimensions we are limiting it to
embedding_matrix

How is this different from previous preprocessing steps?

- 


In [ ]:
# Time to model!
model = keras.models.Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=max_length, 
                           trainable=False)) # Note - not retraining the embedding layer
model.add(layers.Flatten()) # flattening these layers down before connecting to dense layer
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train_processed, y_train,
                    epochs=10,
                    batch_size=100,
                    validation_data=(X_test_processed, y_test))

In [ ]:
score = model.evaluate(X_test_processed, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Visualize results
visualize_training_results(history)

Evaluate:

- Super overfit and not better than our Naive Bayes. This tells us we don't need a NN for this problem


### Treat Embeddings as Starting Weights, but Allow Training:

In [ ]:
# Time to model!
model = keras.models.Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=max_length, 
                           trainable=True)) # Now it can retrain the embedding layer
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train_processed, y_train,
                    epochs=10,
                    batch_size=100,
                    validation_data=(X_test_processed, y_test))
# Takes about... 3 minutes?

In [ ]:
score = model.evaluate(X_test_processed, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Visualize results
visualize_training_results(history)

Evalutate:

- 


### Early Stopping

Patience: how many epochs that model can keep running without improvement before the training is stopped

Reference: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

In [ ]:
# Implement early stopping
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

*If we don't know how many epochs to run, use EarlyStopping*

In [ ]:
# Combine with a model saving feature, so it saves as it improves
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
# Same model as just before this
model.summary()

In [ ]:
# Just adding more epochs
history = model.fit(X_train_processed, y_train,
                    epochs=100,
                    batch_size=100,
                    validation_data=(X_test_processed, y_test),
                    callbacks=[es, mc])

# This takes... a while

### LSTM

Note: there might still be a bug in tensorflow related to the newest numpy version, if you have numpy version 1.20+ this might not work.

https://github.com/tensorflow/models/issues/9706

*Really good for Time-Series and figuring out what the next word should be*

In [ ]:
np.__version__

In [ ]:
model = keras.models.Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim,
                           weights=[embedding_matrix],
                           input_length=max_length,
                           trainable=True))
# Replacing our Flattening layer with an LSTM
# Adding some dropout to prevent overfitting - note the two ways to do so
model.add(layers.LSTM(embedding_dim, 
                      dropout=0.2,
                      return_sequences=True))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

We could do this all here... or we could move over to Kaggle and run this with GPUs!

### Saving your model

*These steps are good for deploying your model*

In [ ]:
model.save('model.h5')
model.save_weights('model_weights.h5')

In [ ]:
from keras.models import load_model

my_model = load_model('model.h5')
my_model.load_weights('model_weights.h5')

In [ ]:
my_model.evaluate(X_test_processed, y_test)

## Additional Resources

- Sklearn's [Working with Text Data Tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

What else can we do with natural language data beyond text classification? 

- [This blog post](https://blog.aureusanalytics.com/blog/5-natural-language-processing-techniques-for-extracting-information) by Aureus Analytics provides an overview of other machine learning techniques used to extract meaning from text: Named Entity Recognition, Sentiment Analysis, Text Summarization, Aspect Mining and Topic Modeling

### Neural Network Vectorizer Resources:

Want another way to embed words for machine learning? Check out Word2Vec - a way of vectorizing text that tries to capture the relationships between words. See the image below, from [this paper](https://arxiv.org/pdf/1310.4546.pdf) from Google developers, that introduced a Skip-gram neural network model that's been utilized by Word2Vec (which is a tool you can use to implement this model). You'll note that the distance between each country and it's capital city is about the same - that distance actually has meaning, and thus you can imagine that the difference between `cat` and `kitten` would be the same as the difference between `dog` and `puppy`. Et cetera!

![screenshot from a paper on the Skip-gram model from devleopers at Google, https://arxiv.org/pdf/1310.4546.pdf](images/Fig2-DsitributedRepresentationsOfWordsAndPhrasesAndTheirCompositionality.png)

- [Pathmind's A.I. Wiki - A Beginner's Guide to Word2Vec](https://wiki.pathmind.com/word2vec)
- [Chris McCormick's Word2Vec Tutorial](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
- [What is the difference between Word2Vec and GloVe?](https://machinelearninginterview.com/topics/natural-language-processing/what-is-the-difference-between-word2vec-and-glove/)

-----

# Part 2: Image Classification

In [ ]:
# New dataset!
from keras.datasets import cifar10

# load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# summarize loaded dataset
print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (X_test.shape, y_test.shape))
# plot first few images
for i in range(9):
    # define subplot
    plt.subplot(330 + 1 + i)
    # plot raw pixel data
    plt.imshow(X_train[i], cmap=plt.get_cmap('gray'))
    
plt.show()
# If you get a "downloading data" thing, worry not, shouldn't take long

In [ ]:
# Checking the class balance of our data
pd.DataFrame(y_train)[0].value_counts()

*Balanced classes (i.e. Airplane, automobile, bird, etc.)*

In [ ]:
# Still need to prep our data
# Scale images to the [0, 1] range - max is 255
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255
# Make sure images have shape (32, 32, 3)
print("X_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")

### So - What is our Input Shape? What is our Output Shape?

In [ ]:
input_shape = (32, 32, 3) # Size of each image - 32x32 for 3 layers
output_shape = 10 # Number of classes, aka number of possible targets (multi class)

In [ ]:
y_train

*If you left y_train like this, it would try and solve as a regression problem <br> So use `to_categorical` to OneHotEncode*

In [ ]:
# We need to prep our outputs
# Different from binary classification!
y_train = keras.utils.to_categorical(y_train, output_shape)
y_test = keras.utils.to_categorical(y_test, output_shape)

print("y_train shape:",y_train.shape)

In [ ]:
y_train[0]

*Notice how the first target is a 6 and now there is a 1 in the 6th column*

## First - Simple Multi-Layer Perceptron!

*The following in instantiating the model as a list, unlike the `.add()` that we did on the NN checkpoint this morning<br> One thing about NN is that there are so many ways to do the same thing. Just be careful to ask yourself "Do I need this person's code example" or "How can I repurpose this code example for my code"*

In [ ]:
# Model building in a single list
model = keras.Sequential(
    [
        keras.Input(shape=input_shape), # Don't always need this input separately
        layers.Flatten(), # need to flatten our images to be one long array
        layers.Dense(64, activation="tanh"),
        layers.Dense(output_shape, activation="softmax"),
    ])

model.summary()

*We use a softmax because it's a multiclass target/problem*

In [ ]:
batch_size = int(X_train.shape[0]/20) #divide the number of rows in our X_train by 20 (just because)
epochs = 15

# Compiling our model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
batch_size

*Lindsey got better results when she changed the batch size to `int(X_train.shape[0]/100)`<br> Therefore `batch_size = 500`<br> This will take longer but give your model more time to adjust the parameters*

In [ ]:
# Fit our model (save output to a history variable)
history = model.fit(X_train, 
                    y_train, 
                    batch_size=batch_size,
                    epochs=epochs, 
                    validation_data=(X_test, y_test))

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

visualize_training_results(history)

Evaluate:

- Not Good
- I could give it more epochs because the loss plot isn't trailing off yet


## Building a CNN - AKA Incorporating Convolutional Layers

A convolutional neural network is a neural network with **convolutional layers**. CNNs are mainly used for image recognition/classification. They can be used for video analysis, NLP (sentiment analysis, topic modeling), and speech recognition. 

### How do our brains see an image? 

We might see some fluffy tail, a wet nose, flappy ears, and a good boy and conclude we are probably seeing a dog. There is not one singular thing about a dog that our brain recognizes as a dog but an amalgamation of different patterns that allow us to make a probable guess.  

<img src='images/chihuahua.jpeg'/>

### How do computers see images?

<img src='images/architecture.jpeg' width=700/>

To computers, color images are a 3D object - composed of 3 matrices - one for each primary color that can be combined in varying intensities to create different colors. Each element in a matrix represents the location of a pixel and contains a number between 0 and 255 which indicates the intensity of the corresponding primary color in that pixel.

<img src='images/rgb.png'/>

## Convolutions

**To *convolve* means to roll together**. CNNs make use of linear algebra to identify patterns using the pixel values (intensity of R,G, or B). By **taking a small matrix and moving it across an image and multiplying them together every time it moves**, our network can mathematically identify patterns in these images. This small matrix is known as a *kernel* or *filter* and each one is designed to identify a particular pattern in an image (edges, shapes, etc.)

<img src='images/convolve.gif' width=500/>

When a filter is "rolled over" an image, the resulting matrix is called a **feature map** - literally a map of where each pattern of feature is in the image. Elements with higher values indicate the presence of a pattern the filter is looking for. The values (or weights) of the filter are adjusted during back-propagation.

#### Convolutional layer parameters

1. Padding: sometimes it is convenient to pad the input volume with zeros around the border. Helps with detecting patterns at the edge of an image
2. Stride: the number of pixels to shift the filter on each "roll". The larger the stride, the smaller the feature map will be - but we will lose more information


### Pooling Layers

After a convolutional layer, the feature maps are fed into a max pool layer. Like convolutions, this method is applied one patch at a time (usually 2x2). Max pooling simply takes the largest value from one patch of an image, places it in a new matrix next to the max values from other patches, and discards the rest of the information contained in the activation maps. Other methods exist such as average pooling (taking an average of the patch).

<img src='images/maxpool.png'/>

This process results in a new feature map with reduced dimensionality that is then passed into another convolution layer to continue the pattern finding process. These steps are repeated until they are passed to a fully connected layer that proceeds to classify the image using the identified patterns.

### Flattening

Once the neural network has collected a series of patterns that an image contains, it is ready to make a guess as to what the image is. In order to do so, it starts by **flattening** the 2D matrix into a 1D vector, so it can be passed into a normal densely connected layer for classification. Then using this vector, one or many densely connected layers will make a prediction as to what the image is.

Reference for this dataset/NN architecture: https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

*More often, it starts big (128) and goes smaller (32) <br> Andrew said "the 32, 64, 128 is the number of filters. the output shapes are getting smaller like you said"*

In [ ]:
# Model building using "add"

cnn = keras.Sequential()
# We defined a variable input_shape earlier, can use that here
cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn.add(layers.MaxPooling2D((2, 2)))

# now, to get the proper output
cnn.add(layers.Flatten())
cnn.add(layers.Dense(128, activation='relu'))
cnn.add(layers.Dense(10, activation='softmax'))

cnn.compile(loss='categorical_crossentropy',
            optimizer="adam",
            metrics=['accuracy'])

In [ ]:
cnn.summary()

In [ ]:
history = cnn.fit(X_train,
                  y_train,
                  epochs=2, # small since this is just a demo
                  batch_size=batch_size,
                  validation_data=(X_test, y_test))
# note - even with just 2 epochs this'll take at least a few minutes

*If you don't understand what's going on with the model, Welcome to Blackbox Models. You will not be able to explain what's going on. That's ok becuase you don't want to use a NN for explainability. <br> If you really wanted to see what was happening, use LIME*

*You can also go to this github for reference: https://github.com/marcotcr/lime*

In [ ]:
# Evaluate!
score = cnn.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Visualize results
visualize_training_results(history)

A worked-through example on this dataset: https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

### Using Pre-Trained Layers

A pretrained network (also known as a convolutional base for CNNs) consists of layers that have already been trained on typically general data. For images, these layers have already learned general patterns, textures, colors, etc. such that when you feed in your training data, certain features can immediately be detected. This part is **feature extraction**.

You typically add your own final layers to train the network to classify/regress based on your problem. This component is **fine tuning**

Here are the pretrained image classification models that exist within Keras: https://keras.io/api/applications/

VGG19: https://keras.io/api/applications/vgg/#vgg19-function

*This is giving pre-trained weights at the start, giving it a better starting point*

In [ ]:
#from keras.applications import VGG19 #Lindsey is using this because she can
from keras.applications.vgg19 import VGG19 #Maybe this will work better

In [ ]:
pretrained = VGG19(weights='imagenet',
                   include_top=False, # Allows us to set input shape
                   input_shape=input_shape) 
# May download data at this step, shouldn't take long

*If `include_top=True` you need to resize the pictures*

In [ ]:
pretrained.summary()

In [ ]:
cnn = keras.models.Sequential()
cnn.add(pretrained)

# freezing layers so they don't get re-trained with your new data
for layer in cnn.layers:
    layer.trainable=False 

*She is freezing it (`layer.trainable=False`) so that it doesn't change the weights for each layer*

In [ ]:
# adding our own dense layers
cnn.add(layers.Flatten())
cnn.add(layers.Dense(128, activation='relu'))
cnn.add(layers.Dense(1, activation='softmax'))

In [ ]:
cnn.summary()

In [ ]:
# to verify that the weights are "frozen" 
for layer in cnn.layers:
    print(layer.name, layer.trainable)

With this you can now compile and fit your model!

In [ ]:
cnn.compile(loss='categorical_crossentropy',
            optimizer="adam",
            metrics=['accuracy'])

In [ ]:
history = cnn.fit(X_train,
                  y_train,
                  epochs=5, 
                  batch_size=64,
                  validation_data=(X_test_resized, y_test))